# Modelo de Baseline Supervisionado (SL) - Previsão de Lucro

**Objetivo:** Treinar um modelo de baseline (LGBM Regressor) para prever o `Lucro_Real` com base no contexto da campanha e nas features do produto digital.

**Metodologia:**
1.  **Carregar Artefatos:** Carrega o dataset (`.csv`) e os pré-processadores (`.joblib`) gerados pelo `Generator_NEW.py`.
2.  **Preparar Dados:** Define `X` (features) e `y` (alvo) e aplica os pré-processadores carregados (OHE, Scaler).
3.  **Treinar Modelo:** Divide os dados em treino/teste e treina um `LGBMRegressor`.
4.  **Avaliar (Global):** Calcula as métricas globais (MAE, RMSE, R²).
5.  **Avaliar (Granular):** Analisa o desempenho do modelo por segmentos (ex: `Tier`, `Tipo_Produto`).
6.  **Explicar (SHAP):** Usa SHAP para entender quais features mais impactam a previsão de lucro.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb
import shap
import d3rlpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

print("Carregando artefatos gerados pelo Generator...")

df_sl = pd.read_csv("sl_dataset_combined.csv")
print(f"✓ Dataset 'sl_dataset_combined.csv' carregado ({len(df_sl)} linhas)")

encoder_sl = joblib.load("sl_encoder.joblib")
print("✓ Encoder 'sl_encoder.joblib' carregado")

scaler_estado = joblib.load("sl_scaler_estado.joblib")
print("✓ Scaler 'sl_scaler_estado.joblib' carregado")

scaler_lucro = joblib.load("sl_scaler_lucro.joblib")
print("✓ Scaler 'sl_scaler_lucro.joblib' carregado")


In [ ]:
target_column = "Lucro_Real"

X_raw = df_sl.drop(columns=[target_column])
y_raw = df_sl[[target_column]]

print(f"Alvo (y): {target_column}")
print(f"Total de features de entrada (X): {X_raw.shape[1]}")


## 2. Preparar Dados (Definir X e y, Pré-processar)

In [ ]:
print("\nAplicando pré-processamento (encoder + scaler de estado)...")

X_encoded = encoder_sl.transform(X_raw)
X_processed = scaler_estado.transform(X_encoded)

y_processed = scaler_lucro.transform(y_raw.values.reshape(-1, 1))

print(f"Formato final de X_processed: {X_processed.shape}")
print(f"Formato final de y_processed: {y_processed.shape}")

# nomes pós-OHE para SHAP:
feature_names_processed = encoder_sl.get_feature_names_out()


## 3. Treinar Modelo (LGBMRegressor)

In [ ]:
print("\nAplicando pré-processamento (encoder + scaler de estado)...")

X_encoded = encoder_sl.transform(X_raw)
X_processed = scaler_estado.transform(X_encoded)

y_processed = scaler_lucro.transform(y_raw.values.reshape(-1, 1))

print(f"Formato final de X_processed: {X_processed.shape}")
print(f"Formato final de y_processed: {y_processed.shape}")

# nomes pós-OHE para SHAP:
feature_names_processed = encoder_sl.get_feature_names_out()


## 4. Avaliar (Global)

Calcula as métricas e reverte o scaling para R$ (conforme snippet)

In [ ]:
def analyze_segments(segment_col: str,
                     X_test_df_orig: pd.DataFrame,
                     y_true_real: np.ndarray,
                     y_pred_real: np.ndarray):
    print(f"\n--- Métricas por Segmento: {segment_col} ---")

    if segment_col not in X_test_df_orig.columns:
        print(f"AVISO: Coluna '{segment_col}' não encontrada. Pulando.")
        return

    df_eval = X_test_df_orig.copy()
    df_eval["y_true_real"] = y_true_real.flatten()
    df_eval["y_pred_real"] = y_pred_real.flatten()

    for segment_value in df_eval[segment_col].unique():
        group = df_eval[df_eval[segment_col] == segment_value]

        mae_seg = mean_absolute_error(group["y_true_real"], group["y_pred_real"])
        rmse_seg = mean_squared_error(group["y_true_real"], group["y_pred_real"], squared=False)
        r2_seg = r2_score(group["y_true_real"], group["y_pred_real"])

        print(f"\nSegmento: {segment_col} = {segment_value} ({len(group)} amostras)")
        print(f"  MAE (R$): {mae_seg:,.2f}")
        print(f"  RMSE (R$): {rmse_seg:,.2f}")
        print(f"  R²: {r2_seg:.2%}")

print("="*30)
print("MÉTRICAS GRANULARES (Baseline SL)")
print("="*30)

analyze_segments("Product_Tier", X_test_orig_df, y_true_real, y_pred_real)
analyze_segments("Offer_Type", X_test_orig_df, y_true_real, y_pred_real)
analyze_segments("Pricing_Model", X_test_orig_df, y_true_real, y_pred_real)


## 5. Avaliar (Granular por Segmento)

Análise de desempenho granular, conforme `prompt + alterações (1).pdf`.

In [ ]:
print("\nCalculando valores SHAP para explicabilidade...")

explainer = shap.TreeExplainer(lgbm_model)
shap_values = explainer.shap_values(X_test)

print("✓ Valores SHAP calculados. Gerando gráfico de resumo...")
shap.summary_plot(shap_values, X_test, feature_names=feature_names_processed)


## 6. Explicar (SHAP)

Análise de importância das features, conforme `prompt + alterações (1).pdf`.

In [ ]:
print("\nCalculando valores SHAP para explicabilidade...")

# 1. Criar o explicador
explainer = shap.TreeExplainer(lgbm_model)

# 2. Calcular os valores SHAP (usamos X_test processado)
shap_values = explainer.shap_values(X_test)

print("✓ Valores SHAP calculados. Gerando gráfico de resumo...")

# 3. Gerar o gráfico de resumo
# Note: Passamos X_test (processado) e os nomes das features (pós-processamento)
shap.summary_plot(shap_values, X_test, feature_names=feature_names_processed)